## <img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>Using the image cutout tool with DP0.2</b> <br>
Contact author(s): <i>Christina Williams</i> <br>
Last verified to run: <i>2024-1-29</i> <br>
LSST Science Piplines version: Weekly <i>2023_47</i> <br>
Container Size: <i>medium</i> <br>
Targeted learning level: <i>beginner</i> <br>

In [1]:
#%load_ext pycodestyle_magic
#%flake8_on
#import logging
#logging.getLogger("flake8").setLevel(logging.FATAL)

**Description:** A demo of new search query behavior after bumping the recommended image to Weekly 2024 04 

In [2]:

# Import general python packages
import time
import numpy as np

# Science Pipelines imports
from lsst.daf.butler import Butler, DatasetType, CollectionType
import lsst.geom as geom
import lsst.resources
import lsst.afw.image as afwImage
from lsst.afw.image import Exposure, ExposureF

# Pandas tools for working with tables
import pandas
from pandas.testing import assert_frame_equal

# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service, retrieve_query

# Astropy
from astropy.coordinates import SkyCoord
from astropy.visualization import ZScaleInterval, AsinhStretch
from astropy.wcs import WCS


### 1.2 Define Functions and Parameters


#### 1.2.1 Set up  plotting defaults for matplotlib.

In [3]:

# Set the maximum number of rows to display from pandas
pandas.set_option('display.max_rows', 20)


In [4]:
service = get_tap_service()

/tmp/ipykernel_19464/1920553455.py:1: DeprecationWarning: get_tap_service() is deprecated, use get_tap_service("tap")
  service = get_tap_service()


First, define a point on the sky as the center of the image cutout. This example uses the galaxy cluster from DP0.3 Notebook Tutorial 03a. Once the RA and Dec are defined, we will create a SpherePoint class to define the location on the sky, that is then used to identify which Patch and Tract contains that location among the deepCoadd images.  

In [5]:
center_coords = "62, -37"

radius = "0.5"


In [6]:

query = "SELECT TOP 5000 objectId, coord_ra, coord_dec, " + \
        "u_ap09Flux, " + \
        "u_ap09Flux/u_ap09FluxErr as ubandS2N "\
        "FROM dp02_dc2_catalogs.Object " + \
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), " + \
        "CIRCLE('ICRS', " + center_coords + ", " + radius + ")) = 1 " + \
        "AND detect_isPrimary = 1 AND detect_fromBlend = 0 "

results = service.search(query)

results.to_table()

objectId,coord_ra,coord_dec,u_ap09Flux,ubandS2N
,deg,deg,nJy,
int64,float64,float64,float64,object
1651413688361432675,61.9683169,-36.5458677,129.8071105,1.8820044580909516
1651413688361432674,61.8607103,-36.5458751,24.7461712,0.3358382441578471
1651413688361432673,61.8820737,-36.5460743,5.3881203,0.07711308436721988
1651413688361432672,61.838656,-36.5459129,37.4895647,0.5456719779976915
1651413688361432669,61.9169532,-36.5468863,-27.8402983,-0.3686051493357243
1651413688361432668,61.845632,-36.5470106,-58.8446558,-0.8175252708028184
1651413688361432676,61.7834028,-36.5453128,115.8229579,1.6794367064897746
1651413688361432677,61.7997912,-36.5452765,-27.747789,-0.42210682049868475


In [7]:
#  here, figure out where u-band flux is less then required S/N

whnondet = np.where(results['ubandS2N'] < 3)[0]


TypeError: '<' not supported between instances of 'str' and 'int'